<a href="https://colab.research.google.com/github/liranbd1/Deep_Learning_HW2/blob/main/Deep_Learning_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTE!!!**

In this code there are hard coded paths to Google Drive which load the dataset, write files and create folder, please make sure to change the path parameters to avoid this if not wanted.

In [31]:
import os
import torchvision
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn as nn
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk


**Connecting to Google Drive**

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Custom Dataset class**

Custom dataset class to load image files with their labels.

In [33]:
class GTSRB(Dataset):
    def __init__(self, root, csv, transform = False, shuffle = False):
        self.root_dirc = root
        self.df = pd.read_csv(os.path.join(root, csv))
        self.transform = transform
        if shuffle:
            self.df = self.df.sample(frac=1)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_row = self.df.iloc[index]
        img_path = os.path.join(self.root_dirc, img_row['Path'])

        image = Image.open(img_path)
        label = img_row['ClassId']

        if self.transform:
            image = self.transform(image)

        return image, label

**Loading Dataset**

First block will be loading datasets from google drive

In [73]:
transform = transforms.Compose(
    [
        transforms.Resize((30,30)),
        transforms.ToTensor(),
        transforms.Normalize((0.34, 0.31, 0.31),(0.27,0.26,0.26))
    ]
)

def load_datasets():
    data_path = os.path.join(root_path, 'datasets')
    train_csv_filename = 'Train.csv'
    test_csv_filename = 'Test.csv'

    # Loading datasets 
    train_dataset = GTSRB(data_path, train_csv_filename, transform, shuffle = True)
    test_dataset = GTSRB(data_path, test_csv_filename, transform)

    # Splitting train to train and validation.
    train_len = int(0.8*len(train_dataset))
    val_len = len(train_dataset) - train_len

    train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

    # Creating dataloaders
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, num_workers=2)
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64, num_workers=2)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, num_workers=2)

    return train_dataloader, validation_dataloader, test_dataloader

**Hyper Parameters**


In [35]:
# Setting the device to Cuda if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Output and input layers
output_layer_size = 43
input_layer_size = 3

# Hyper parameters for NoDropNoBNCNN
NoDropNoBN_kernel = 3
NoDropNoBN_strid = 1
NoDropNoBN_padding = 1

# Early stopping parameter
p = 20

# Optimizers


# Learning rates
learning_rate = 0.001

# Paths
root_path = './gdrive/MyDrive/Deep_Learning_HW2'


In [36]:
# DataLoaders
train_dl, validation_dl, test_dl = load_datasets()



In [44]:
print(device)

cuda


**Model 1**

No dropout and no batch normalization

In [77]:
class NoDropNoBNCNN(nn.Module):
  def __init__(self, in_channels):
    super().__init__()
    self.feature_extractor = nn.Sequential(
      nn.Conv2d(in_channels, out_channels=32, kernel_size=3, stride=1, padding= 1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(32, out_channels=64, kernel_size=3, stride=1, padding= 1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(64, 64, 3,stride=1, padding=1 ),
      nn.MaxPool2d(2)
    )
    #Add another Convolution layer or reduce the number of feature maps

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(576, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, output_layer_size)
    )
    self.softmax = torch.nn.Softmax(dim=1)

  def forward(self, x):
    features = self.feature_extractor(x)
    #print('x:', x.shape)
    #print('features: ', features.shape)
    class_scores = self.classifier(features)
   
    #print('class_scores: ', class_scores.shape)
    y_pred = self.softmax(class_scores)
    return y_pred

In [78]:
def training(epochs, optimizer, model, loss_func, patience, model_type):
  patience_count = 0
  model.train()
  train_acc_list = []
  val_acc_list = []
  best_model_path = os.path.join(root_path, f'best_model{model_type}.pth')
  for i in range(epochs):
    for idx, (data, label) in enumerate(train_dl):
      print(f'data index {idx}')
      optimizer.zero_grad()
      y_hat = model(data.to(device))
      loss = loss_func(y_hat, label.to(device))
      loss.backward()
      optimizer.step()
    
    # Calculate Accuracy for train and validation
    train_acc_list.append(calculate_accuracy(train_dl, model))
    val_acc_list.append(calculate_accuracy(validation_dl, model))

    model.eval()
    with torch.no_grad():
      for j, (data, label) in enumerate(validation_data):
          optimizer.zero_grad()
          y_hat = model(data.to(device))
          loss = loss_function(y_hat, label.to(device))
          optimizer.step()
      loss_val = loss.detach()
      if loss_val < minimal_loss:
        minimal_loss = loss_value
        patience_count = 0
        torch.save(model.state_dict(), best_model_path)
      elif loss_val >= minimal_loss:
        patience_count += 1
    
    if patience_count == patience:
      break

  return train_acc_list, val_acc_list


In [39]:
def calculate_accuracy(dataloader, model, title=None):
  correct_count = 0
  total_count = 0
  
  for j, (data,label) in enumerate(data_loader):
    y_hat = model(data)
    predictions = torch.argmax(y_hat, dim=1)
    correct_count += torch.sum(predictions == label.to(device)).type(torch.float32)
    total_count += data.shape[0]
  accuracy = (correct_count/ total_count).item()*100
  
  # This section is relevant to the test accuracy
  if title != None:
    # Checking if path exists and if not create it
    if not os.path.exists('./gdrive/MyDrive/Deep_Learning_HW2/accuracy_file'):
      try: # Since we can throw an error here we prefer to wrap it in try and catch and raise the error back to not continue the run
        !mkdir './gdrive/MyDrive/Deep_Learning_HW2/accuracy_file'
      except OSError as error:
        print(error)
        raise error 
    # Writing the accuracy to a file
    with open(os.path.join('./gdrive/MyDrive/Deep_Learning_HW2/accuracy_file',f'{line_title}.txt'), 'w') as file:
      file.write(f"accuracy : {accuracy} %")

  return accuracy

In [40]:
def plot_report(x_axis, title, y_axis):
  
  file_name = title
  if not os.path.exists('./gdrive/MyDrive/Deep_Learning_HW2/plots'):
      try: # Since we can throw an error here we prefer to wrap it in try and catch and raise the error back to not continue the run
        !mkdir './gdrive/MyDrive/Deep_Learning_HW2/plots'
      except OSError as error:
        print(error)
        raise error 

  fig = plt.figure()
  plt.plot(x_axis, y_axis, 'b')
  plt.title(title)
  file_path = os.path.join('./gdrive/MyDrive/Deep_Learning_HW2/plots', f'{file_name}')
  fig.savefig(file_path)

In [ ]:
model = NoDropNoBNCNN(3).to(device)
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
loss_function = torch.nn.CrossEntropyLoss()
epochs = 5
train_acc, val_acc = training(epochs, optimizer, model, loss_function, 5, 'No_D_No_BN')

plot_report(train_acc, "No_D_No_BN train accuracy", epochs)
plot_report(val_acc, "No_D_No_BN validation accuracy", epochs)

test_acc = calculate_accuracy(test_dl, model, 'No_d_No_BN test accuracy')

print(test_acc)

data index 0
data index 1
data index 2
data index 3
data index 4
data index 5
data index 6
data index 7
data index 8
data index 9
data index 10
data index 11
data index 12
data index 13
data index 14
data index 15
data index 16
data index 17
data index 18
data index 19
data index 20
data index 21
data index 22
data index 23
data index 24
data index 25
data index 26
data index 27
data index 28
data index 29
data index 30
data index 31
data index 32
data index 33
data index 34
data index 35
data index 36
data index 37
data index 38
data index 39
data index 40
data index 41
data index 42
data index 43
data index 44
data index 45
data index 46
data index 47
data index 48
data index 49
data index 50
data index 51
data index 52
data index 53
data index 54
data index 55
data index 56
data index 57
data index 58
data index 59
data index 60
data index 61
data index 62
data index 63
data index 64
data index 65
data index 66
data index 67
data index 68
data index 69
data index 70
data index 71
da